# First convolutions

## Objectives :
  - Recognizing traffic signs 
  - Understand the **principles** and **architecture** of a **convolutional neural network** for image classification
  
The German Traffic Sign Recognition Benchmark (GTSRB) is a dataset with more than 50,000 photos of road signs from about 40 classes.  
The final aim is to recognise them ! 

# Step 1 - Import and init

## 1.1 - Python stuff

In [3]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

import keras

import numpy as np
import matplotlib.pyplot as plt
import h5py
import os,time,sys

from importlib import reload

from sklearn.utils import shuffle

### 1.2 - Parameters
`scale` is the proportion of the dataset that will be used during the training. (1 mean 100%)  
A 20% 24x24 dataset, with 5 epochs and a scale of 1, need  **3'30** on a CPU laptop.\
`fit_verbosity` is the verbosity during training : 0 = silent, 1 = progress bar, 2 = one line per epoch

In [4]:
enhanced_dir = './data'
# enhanced_dir = f'{datasets_dir}/GTSRB/enhanced'

dataset_name  = 'set-24x24-L'
batch_size    = 64
epochs        = 5
scale         = 1
fit_verbosity = 1

Override parameters (batch mode) - Just forget this cell

## Step 2 - Load dataset
We're going to retrieve a previously recorded dataset.  
For example: set-24x24-L

In [5]:
def rescale_dataset(x_train, y_train, x_test, y_test, scale=1):
    n_train = int(len(x_train) * scale)
    n_test = int(len(x_test) * scale)

    return x_train[:n_train], y_train[:n_train], x_test[:n_test], y_test[:n_test]

In [6]:
def read_dataset(enhanced_dir, dataset_name, scale=1):
    '''
    Reads h5 dataset
    Args:
        filename     : datasets filename
        dataset_name : dataset name, without .h5
    Returns:    
        x_train,y_train, x_test,y_test data, x_meta,y_meta
    '''
    
    start_time = time.time()

    filename = f'{enhanced_dir}/{dataset_name}.h5'
    with h5py.File(filename, 'r') as f:
        x_train = f['x_train'][:]
        y_train = f['y_train'][:]
        x_test = f['x_test'][:]
        y_test = f['y_test'][:]
        x_meta = f['x_meta'][:]
        y_meta = f['y_meta'][:]

    print('Original shape :', x_train.shape, y_train.shape)
    x_train, y_train, x_test, y_test = rescale_dataset(x_train, y_train, x_test, y_test, scale=scale)
    print('Rescaled shape :', x_train.shape, y_train.shape)

    x_train, y_train = shuffle(x_train, y_train, random_state=42)

    duration = time.time() - start_time
    size = os.path.getsize(filename)
    print(f'\nDataset "{dataset_name}" is loaded and shuffled. ({size} bytes in {duration:.2f} seconds)')
    return x_train, y_train, x_test, y_test, x_meta, y_meta


# ---- Read dataset
#
x_train,y_train,x_test,y_test, x_meta,y_meta = read_dataset(enhanced_dir, dataset_name, scale)

Original shape : (7841, 24, 24, 1) (7841,)
Rescaled shape : (7841, 24, 24, 1) (7841,)

Dataset "set-24x24-L" is loaded and shuffled. (24270160 bytes in 0.04 seconds)


## Step 3 - Have a look to the dataset
We take a quick look as we go by...

In [ ]:
import matplotlib.pyplot as plt

def display_and_save_images(images, labels, indices, columns, x_size, y_size, save_as=None):
    
    rows = len(indices) // columns + (1 if len(indices) % columns else 0)
    fig, axes = plt.subplots(rows, columns, figsize=(x_size * columns, y_size * rows))
    axes = axes.flatten()

    for ax, idx in zip(axes, indices):
        ax.imshow(images[idx], cmap='gray', interpolation='nearest')  # Adjust colormap if needed
        ax.set_title(f'Label: {labels[idx]}')
        ax.axis('off')

    for i in range(len(indices), len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    if save_as:
        plt.savefig(f'{save_as}.png', dpi=300)
    plt.show()
